In [13]:
import igl

import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from math import exp
import quaternion

from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [47]:
def similarity(Wp, Wv, sigma=0.1):
    #BE shape
    Wp = Wp if len(Wp.shape) == 1 else Wp.reshape(-1)
    Wv = Wv if len(Wv.shape) == 1 else Wv.reshape(-1)
    
    tot = 0
    for j in range(Wp.shape[0]):
        for k in range(j+1, Wv.shape[0]):
            tot += Wp[j]*Wp[k]*Wv[j]*Wv[k]*exp( -(Wp[j]*Wv[k] - Wp[k]*Wv[j])**2 / sigma**2)
            
    return tot

def CoR(i, weights, vertices, faces, omega=0.1, pb = None):
    
    num = np.zeros([1, 3])
    denom = np.zeros([1,3])
    for t in range(faces.shape[0]):
        # cmp = [i for i in range(3) if np.linalg.norm(weights[i] - weights[faces[t,i]]) < omega]
        cmp = [0, 1,2]
        if not cmp:
            continue
        s = similarity(weights[i], np.mean([weights[faces[t,c]] for c in cmp], axis = 0))
        v = np.mean([vertices[faces[t,c]] for c in cmp], axis = 0)
        a = igl.doublearea(vertices, faces[[t]]) / 2 

        num += s * v * a
        denom += s * a
    
    pi = num / denom
    
    if pb:
        pb.value = i

    return pi

In [14]:
V, F = igl.read_triangle_mesh('data/cloth_ball0.ply')

In [15]:
_, V, F, _, _= igl.decimate(V, F, 5000)

In [17]:
plane_surf_F = F[igl.face_components(F) == 0]
plane_surf_V = V[list(set(plane_surf_F.reshape(-1))), :]
mp.plot(plane_surf_V, plane_surf_F, shading={"wireframe" : True})

/Users/davidbelgrod/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.009898…

In [42]:
C = []
mn = np.min(plane_surf_V,axis=0)
for i in range(0, 60, 20):
    for j in range (0, 60, 20):
        C.append(mn + [i, 0, j])
C = np.array(C)
p = mp.plot(plane_surf_V,plane_surf_F)
p.add_points(C, shading={"point_size": 4.0})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.009898…

1

In [ ]:
# bd = igl.boundary_loop(plane_surf_F)
# bd = bd[::11] 
# C = plane_surf_V[bd,:]
# ctr = np.median(V, axis=0)
# C = np.concatenate((C, [ctr]), axis=0)

# p = mp.plot(plane_surf_V,plane_surf_F)
# p.add_points(C, shading={"point_size": 4.0})
P = np.array(range(C.shape[0]))
_, b, bc = igl.boundary_conditions(plane_surf_V,plane_surf_F, C, P , np.empty((0,2),dtype='int64'), np.empty((0,2),dtype='int64'))
bbw_solver = igl.BBW()
bbw_solver.solve(plane_surf_V, plane_surf_F, b ,bc)

ValueError: Invalid dimension for argument v_init. Must have shape (#vertices, 2) for triangle mesh inputs. You passed in V with shape (2258, 3)

# https://github.com/libigl/libigl-python-bindings/commit/bc3bb8cb6273b673d2690859a5da8d7092e96282

In [45]:
# wj(x0) = d(x0, Hj)^-1
W = np.array([1 / np.linalg.norm(plane_surf_V - C[i], axis=1) for i in range(C.shape[0])])
W = (W.T / np.linalg.norm(W, axis=0)[:,None])
mp.plot(plane_surf_V, plane_surf_F, c = W[:,4])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.009898…

In [48]:
V = plane_surf_V
F = plane_surf_F

P = np.zeros(V.shape)

with tqdm_joblib(tqdm(desc="My calculation", total=P.shape[0])) as progress_bar:
    P = Parallel(n_jobs=10)(delayed(CoR)(i, W, V, F) for i in range(P.shape[0]))
P = np.array(P)

My calculation: 100%|███████████████████████| 2258/2258 [01:56<00:00, 19.36it/s]


In [49]:
ok = mp.plot(V,F)
ok.add_points(P, shading={"point_size": 1.0, 'coloring' : 'green'})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.009898…

1

In [ ]:
# pickle.dump(P, open('data/CoR_cloth_ball.p', 'wb'))